In [4]:
import pyspark
import findspark
import pandas as pd

In [217]:
data = pd.read_csv("../data/Main_Last.csv")
data['DateFull'] = pd.to_datetime(data[['Year', 'Month', 'Day']])

In [218]:
data.to_csv('../data/Main_Last.csv', index=False)

In [76]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

In [79]:
data['Count'] = data.groupby(data['Date'].dt.date)['Date'].transform('count')

In [81]:
data_sorted = data.sort_values(by=['Date', 'Count'], ascending=[True, False])

In [86]:
data_sorted = data_sorted.reset_index(drop=True)
data_sorted = data_sorted.drop_duplicates(subset=['Date'])
data_sorted

,Date,Count
0,2016-01-14,2
1,2016-02-08,1
2,2016-02-09,4
3,2016-02-10,5
4,2016-02-11,5
...,...,...
2532,2023-03-27,144
2533,2023-03-28,158
2534,2023-03-29,150
2535,2023-03-30,124


In [91]:
min_date = data['Date'].min()
max_date = data['Date'].max()
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
all_dates_df = pd.DataFrame({'Date': all_dates})

# Thực hiện phép gộp giữa DataFrame mới và DataFrame hiện tại dựa trên cột 'Date'
merged_df = pd.merge(all_dates_df, data_sorted, on='Date', how='left')

# Điền giá trị 0 cho các ngày không có trong DataFrame ban đầu
merged_df['Count'] = merged_df['Count'].fillna(0).astype(int)
merged_df.to_csv("../data/Main_Last.csv")

In [198]:
data.to_csv('../data/Main_Last.csv', index=False)

---

In [5]:
findspark.init()

In [6]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [7]:
spark

In [8]:
input_df = spark.read.csv("../data/Main_Last.csv", header=True, inferSchema=True)

In [9]:
input_df.show()

+---+-----+----+-----+---+----------+
| ID|Count|Year|Month|Day|  DateFull|
+---+-----+----+-----+---+----------+
|  0|    2|2016|    1| 14|2016-01-14|
|  1|    0|2016|    1| 15|2016-01-15|
|  2|    0|2016|    1| 16|2016-01-16|
|  3|    0|2016|    1| 17|2016-01-17|
|  4|    0|2016|    1| 18|2016-01-18|
|  5|    0|2016|    1| 19|2016-01-19|
|  6|    0|2016|    1| 20|2016-01-20|
|  7|    0|2016|    1| 21|2016-01-21|
|  8|    0|2016|    1| 22|2016-01-22|
|  9|    0|2016|    1| 23|2016-01-23|
| 10|    0|2016|    1| 24|2016-01-24|
| 11|    0|2016|    1| 25|2016-01-25|
| 12|    0|2016|    1| 26|2016-01-26|
| 13|    0|2016|    1| 27|2016-01-27|
| 14|    0|2016|    1| 28|2016-01-28|
| 15|    0|2016|    1| 29|2016-01-29|
| 16|    0|2016|    1| 30|2016-01-30|
| 17|    0|2016|    1| 31|2016-01-31|
| 18|    0|2016|    2|  1|2016-02-01|
| 19|    0|2016|    2|  2|2016-02-02|
+---+-----+----+-----+---+----------+
only showing top 20 rows



---

In [10]:
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col
import time

In [11]:
input_df.createOrReplaceTempView("main_table")

In [12]:
last = pd.DataFrame()

In [204]:
input_df.show()

+---+-----+----+-----+---+
| ID|Count|Year|Month|Day|
+---+-----+----+-----+---+
|  0|    2|2016|    1| 14|
|  1|    0|2016|    1| 15|
|  2|    0|2016|    1| 16|
|  3|    0|2016|    1| 17|
|  4|    0|2016|    1| 18|
|  5|    0|2016|    1| 19|
|  6|    0|2016|    1| 20|
|  7|    0|2016|    1| 21|
|  8|    0|2016|    1| 22|
|  9|    0|2016|    1| 23|
| 10|    0|2016|    1| 24|
| 11|    0|2016|    1| 25|
| 12|    0|2016|    1| 26|
| 13|    0|2016|    1| 27|
| 14|    0|2016|    1| 28|
| 15|    0|2016|    1| 29|
| 16|    0|2016|    1| 30|
| 17|    0|2016|    1| 31|
| 18|    0|2016|    2|  1|
| 19|    0|2016|    2|  2|
+---+-----+----+-----+---+
only showing top 20 rows



In [14]:
data = pd.read_csv('../data/Main_Last.csv')

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
def visualize(data):
    plt.plot(data['DateFull'], data['Count'])
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.title('Line Chart of Count over Date')

    # Xoay nhãn trục x nếu cần
    plt.tight_layout()  # Đảm bảo không bị cắt bớt các nhãn
    plt.show()


In [30]:
year = 2016
result = pd.DataFrame()
def update():
    if year < input_df.count():
        for month in range(1, 12):
            sql_query = "SELECT * FROM main_table WHERE Year = '{}' AND Month = '{}'".format(year, month)
            cut_df = spark.sql(sql_query).toPandas()
            result = pd.concat([result, cut_df], axis=0)
        year += 1

In [38]:
year = 2016
result = pd.DataFrame()
def update():
    if year < 2024:
        for month in range(1, 12):
            sql_query = "SELECT * FROM main_table WHERE Year = '{}' AND Month = '{}'".format(year, month)
            cut_df = spark.sql(sql_query).toPandas()
            result = pd.concat([result, cut_df], axis=0)
        year += 1

In [21]:
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource

In [28]:
p = figure(outer_height=300, outer_width=800)
p.line(x="x", y="y", source=result)

GlyphRenderer(id='p1053', ...)

In [29]:
curdoc().add_periodic_callback(update, 100)
curdoc().add_root(p)

In [ ]:
import pandas as pd

# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv("../data/Main_Last.csv")

# Xóa các hàng có giá trị trong cột "Count" là 0
df = df[df["Count"] != 0]

# Lưu DataFrame đã chỉnh sửa lại thành file CSV
df.to_csv("../data/Main_Last_Miss.csv", index=False)
